In [1]:
# https://platform.olimpiada-ai.ro/problems/62

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [2]:
train = pd.read_csv("/kaggle/input/archive-of-drawn-words/train.csv")
test = pd.read_csv("/kaggle/input/archive-of-drawn-words/test.csv")

train.shape, test.shape

((400, 2), (100, 1))

In [3]:
train.head(2)

,image_path,label
0,output_dataset/train/000000.png,violin
1,output_dataset/train/000001.png,moon


In [4]:
train['label'].value_counts()

label
cake        26
star        24
house       24
umbrella    23
violin      23
dog         22
elephant    21
banana      20
cat         20
fish        19
moon        19
lion        19
book        19
apple       19
pencil      18
sun         18
boat        18
flower      17
tree        17
kite        14
Name: count, dtype: int64

In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("zero-shot-image-classification", model="openai/clip-vit-base-patch32")

2025-12-14 14:54:00.650767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765724041.072728      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765724041.195645      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


In [6]:
candidate_labels = [
    'apple', 'banana', 'cat', 'dog', 'elephant',
    'flower', 'house', 'moon', 'sun', 'tree',
    'violin', 'lion', 'kite', 'boat', 'star',
    'fish', 'pencil', 'cake', 'book', 'umbrella'
]

out = pipe(os.path.join('/kaggle/input/archive-of-drawn-words', train['image_path'][0]), candidate_labels=candidate_labels)
choice = out[0]['label']

print(choice, train['label'][0], sep=', ')

violin, violin


In [7]:
correct, total = 0, 0

for i, row in (pbar := tqdm(train.iterrows(), total=len(train))):
    out = pipe(os.path.join('/kaggle/input/archive-of-drawn-words', row['image_path']), candidate_labels=candidate_labels)
    choice = out[0]['label']
    if choice == row['label']:
        correct += 1
    total += 1

    pbar.set_postfix({'accuracy': f'{correct/total:.5f}'})

print(f'Accuracy: {correct/total:.5f}')

  0%|          | 0/400 [00:00<?, ?it/s]

Accuracy: 0.95000


In [8]:
paths, preds = [], []

for i, row in (pbar := tqdm(test.iterrows(), total=len(test))):
    out = pipe(os.path.join('/kaggle/input/archive-of-drawn-words', row['image_path']), candidate_labels=candidate_labels)
    choice = out[0]['label']

    paths.append(row['image_path'])
    preds.append(choice)

subm = pd.DataFrame({
    'image_path': paths,
    'label': preds
})

subm.to_csv("submission.csv", index=False)

subm.head()

  0%|          | 0/100 [00:00<?, ?it/s]

,image_path,label
0,output_dataset/test/000000.png,tree
1,output_dataset/test/000001.png,dog
2,output_dataset/test/000002.png,sun
3,output_dataset/test/000003.png,elephant
4,output_dataset/test/000004.png,tree
